# RENAME GCVI COLS

In [10]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# Load the CSV
df = pd.read_csv("Karnataka_Datasets/Across/Kharif/Davanagere/Davanagere_SAR_NDVI.csv")

# Generate 12-day interval dates from 2019-04-01 to 2020-03-31
start_date = datetime(2019, 4, 1)
end_date = datetime(2020, 3, 31)
interval_dates = [start_date + timedelta(days=12 * i) for i in range(((end_date - start_date).days // 12) + 1)]

def align_band_columns(df, prefix, interval_dates):
    existing_cols = [col for col in df.columns if col.startswith(prefix)]
    existing_date_map = {
        datetime.strptime(col[len(prefix)+1:], "%Y-%m-%d"): col for col in existing_cols
    }

    new_cols = []
    for target_date in interval_dates:
        target_str = target_date.strftime('%Y-%m-%d')
        target_col = f"{prefix}_{target_str}"
        found = False
        for delta in range(-6, 7):  # ±6 day window
            near_date = target_date + timedelta(days=delta)
            if near_date in existing_date_map:
                source_col = existing_date_map[near_date]
                df[target_col] = df[source_col]
                found = True
                break
        if not found:
            df[target_col] = np.nan
        new_cols.append(target_col)

    return sorted(new_cols, key=lambda x: datetime.strptime(x[len(prefix)+1:], "%Y-%m-%d"))

# Align and interpolate
vh_cols = align_band_columns(df, "VH", interval_dates)
vv_cols = align_band_columns(df, "VV", interval_dates)

df[vh_cols] = df[vh_cols].interpolate(axis=1, method='linear', limit_direction='both')
df[vv_cols] = df[vv_cols].interpolate(axis=1, method='linear', limit_direction='both')

# Keep non-VH/VV columns first
other_cols = [col for col in df.columns if not (col.startswith("VH_") or col.startswith("VV_"))]

# Reorder: other + sorted VH + sorted VV
ordered_cols = other_cols + vh_cols + vv_cols
df = df[ordered_cols]

# Save the result
df.to_csv("Karnataka_Datasets/Across/Kharif/Davanagere/Davanagere_SAR_NDVI_Interpolated.csv", index=False)


In [11]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# File path
input_file = "Karnataka_Datasets/Across/Kharif/Davanagere/Davanagere_SAR_NDVI_Interpolated.csv"  # change this if needed

# Load CSV
df = pd.read_csv(input_file)

# Generate 12-day interval dates
start_date = datetime(2019, 4, 1)
end_date = datetime(2020, 3, 31)
interval_dates = [start_date + timedelta(days=12 * i) for i in range(((end_date - start_date).days // 12) + 1)]

def align_band_columns(df, prefix, interval_dates):
    existing_cols = [col for col in df.columns if col.startswith(prefix)]
    existing_date_map = {
        datetime.strptime(col[len(prefix)+1:], "%Y-%m-%d"): col for col in existing_cols
    }

    new_cols = []
    for target_date in interval_dates:
        target_str = target_date.strftime('%Y-%m-%d')
        target_col = f"{prefix}_{target_str}"
        found = False
        for delta in range(-6, 7):  # ±6 day window
            near_date = target_date + timedelta(days=delta)
            if near_date in existing_date_map:
                df[target_col] = df[existing_date_map[near_date]]
                found = True
                break
        if not found:
            df[target_col] = np.nan
        new_cols.append(target_col)

    return sorted(new_cols, key=lambda x: datetime.strptime(x[len(prefix)+1:], "%Y-%m-%d"))

# Process VH and VV
vh_cols = align_band_columns(df, "VH", interval_dates)
vv_cols = align_band_columns(df, "VV", interval_dates)

# Interpolate missing values
df[vh_cols] = df[vh_cols].interpolate(axis=1, method='linear', limit_direction='both')
df[vv_cols] = df[vv_cols].interpolate(axis=1, method='linear', limit_direction='both')

# Rename columns to VH_1...VH_n and VV_1...VV_n
vh_renamed = {col: f'VH_{i+1}' for i, col in enumerate(vh_cols)}
vv_renamed = {col: f'VV_{i+1}' for i, col in enumerate(vv_cols)}
df.rename(columns=vh_renamed, inplace=True)
df.rename(columns=vv_renamed, inplace=True)

# Reorder columns: original non-VH/VV columns + renamed VH + renamed VV
vh_cols_renamed = [vh_renamed[col] for col in vh_cols]
vv_cols_renamed = [vv_renamed[col] for col in vv_cols]
other_cols = [col for col in df.columns if col not in vh_cols_renamed + vv_cols_renamed]
df = df[other_cols + vh_cols_renamed + vv_cols_renamed]

# Overwrite the input file
df.to_csv(input_file, index=False)
